In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
%pip install networkx

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install spektral

Note: you may need to restart the kernel to use updated packages.


In [219]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import glob
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.neighbors import kneighbors_graph
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.feature_extraction import img_to_graph
from tqdm import tqdm
import tensorflow as tf
from torchvision.datasets import DatasetFolder
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Model, regularizers
from spektral.layers import GCNConv
from keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.feature_extraction import image
import spektral
import networkx as nx
from keras.layers import Reshape, Input, Dropout
from keras.optimizers.legacy import Adam

In [ ]:
print(tf.__version__)

In [220]:
#Load dataset path
dataset_path = "Faces"

In [242]:
#Load images path from dataset folder
image_utk = [f for f in glob.glob(dataset_path + "/*.jpg", recursive=True)]

In [298]:
#For each image path, read the image, resize it, create a graph from each image
processed_images= []
for image_path in tqdm(image_utk):
    image = plt.imread(image_path)
    resized_image = cv2.resize(image, (96, 96))
    
    #Create a separate graph for each channel
    if resized_image.shape ==(96,96):
        G = nx.from_numpy_array(resized_image)
        processed_images.append(G)
    else:
      graphs = []
      for channel in range(resized_image.shape[2]):
          gray_channel = resized_image[:, :, channel]
          G = nx.from_numpy_array(gray_channel)
          graphs.append(G)
      # Combine the graphs into a single graph
      combined_graph = nx.compose_all(graphs)
      processed_images.append(combined_graph)
    

100%|██████████| 4521/4521 [00:20<00:00, 217.07it/s]


In [245]:
#Extract age label from image path
labels = []
for image in image_utk:
    labels.append(int(image.split("/")[-1].split("_")[0]))

In [246]:
#Split graphs and labels into training and testing sets
X_train, X_test, y_train,  y_test = train_test_split(processed_images, labels, test_size=0.2, random_state=42)

In [293]:
#Defining the GCNN

channels = 32           # Number of channels in the first layer
dropout = 0.5           # Dropout rate for the features
l2_reg = 5e-4           # L2 regularization rate
learning_rate = 1e-2    # Learning rate
epochs = 100            # Number of training epochs
es_patience = 10

X_in = Input(shape=(96, ))  # Adjust input shape to match the number of nodes
fltr_in = Input((96, ), sparse=True)

dropout_1 = Dropout(dropout)(X_in)
graph_conv_1 = GCNConv(channels,
                         activation='relu',
                         kernel_regularizer=regularizers.l2(l2_reg),
                         use_bias=False)([dropout_1, fltr_in])

dropout_2 = Dropout(dropout)(graph_conv_1)
graph_conv_2 = GCNConv(1,
                         activation='softmax',
                         use_bias=False)([dropout_2, fltr_in])

# Build model
model_2 = Model(inputs=[X_in, fltr_in], outputs=graph_conv_2)
optimizer = Adam(learning_rate=learning_rate)
model_2.compile(optimizer=optimizer,
              loss='mean_absolute_error',
              weighted_metrics=['acc'])
model_2.summary()

Model: "model_27"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_55 (InputLayer)       [(None, 96)]                 0         []                            
                                                                                                  
 dropout_49 (Dropout)        (None, 96)                   0         ['input_55[0][0]']            
                                                                                                  
 input_56 (InputLayer)       [(None, 96)]                 0         []                            
                                                                                                  
 gcn_conv_51 (GCNConv)       (None, 32)                   3072      ['dropout_49[0][0]',          
                                                                     'input_56[0][0]']     

In [247]:
# Compute the adjacency matrices for training set
adjacency_matrices = []
for i in tqdm(X_train):
    adj = nx.adjacency_matrix(i)
    adjacency_matrices.append(adj)


100%|██████████| 3616/3616 [02:33<00:00, 23.52it/s] 


In [249]:
# Compute the adjacency matrices for testing set
adjacency_matrices_test = []
for i in tqdm(X_test):
    adj = nx.adjacency_matrix(i)
    adjacency_matrices_test.append(adj)

100%|██████████| 905/905 [00:51<00:00, 17.44it/s] 


In [ ]:
#adjacency_matrices_np = GCNConv.preprocess(adjacency_matrices_np).astype('f4')

In [154]:
#test = GCNConv.preprocess(adjacency_matrices_np[0]).astype('f4')
#print(test.shape)

(96, 96)


In [261]:
print("Shape of X_train:", X_train[0].shape)
print("Shape of adjacency_matrices_np:", type(adjacency_matrices[0]))
print("Shape of y_train:", y_train[0].shape)

Shape of X_train: (96,)
Shape of adjacency_matrices_np: <class 'numpy.ndarray'>
Shape of y_train: ()


In [265]:
# Assuming adjacency_matrices is a list of csr_matrix objects

# Convert to NumPy array
adjacency_matrices_np = np.array(adjacency_matrices)
adjacency_matrices_np_test = np.array(adjacency_matrices_test)

In [270]:
X_train= np.array(X_train)
X_test= np.array(X_test)

In [289]:
validation_data = ([X_test, adjacency_matrices_np_test], y_test)
gcnn = model_2.fit([X_train, adjacency_matrices_np],
          y_train,
          epochs=epochs,
          batch_size=32,
          validation_data=validation_data,
          shuffle=False,
          callbacks=[
              EarlyStopping(patience=es_patience,  restore_best_weights=True),
          ])

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type csr_array).

In [ ]:
gcnn.save("gcnn.keras")